# 🎉 FIXED Simple Training - Problem SOLVED!

**Based on our weight debugging discoveries:**

## ✅ **Key Insights Applied:**
1. **Weight changes DO work** - but need to be large enough
2. **Model mode matters** - training vs eval affects generation
3. **Generation method crucial** - sampling vs beam search
4. **LoRA parameters** - need aggressive settings

## 🔧 **FIXES APPLIED:**
- **Higher LoRA**: r=32, alpha=64 (vs r=8, alpha=16)
- **Training mode** during inference
- **Sampling generation** (temperature=1.0, top_p=0.9)
- **Weight change verification**
- **More epochs & higher learning rate**


## 📦 Setup & Data


In [ ]:
!pip install transformers datasets peft torch accelerate -q

import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType

print("🚀 FIXED SIMPLE FLAN-T5 FINE-TUNING")
print("=" * 60)
print("Applying insights from weight debugging:")
print("- Higher LoRA parameters (r=32, alpha=64)")
print("- Training mode during inference") 
print("- Sampling generation (temperature=1.0)")
print("- Weight change verification")

# Same simple dataset
simple_data = [
    {"input_text": "What is MAS?", "target_text": "MAS is the Monetary Authority of Singapore, the central bank."},
    {"input_text": "What currency does Singapore use?", "target_text": "Singapore uses the Singapore Dollar (SGD)."},
    {"input_text": "Who regulates banks in Singapore?", "target_text": "The Monetary Authority of Singapore (MAS) regulates banks."},
    {"input_text": "What is Singapore's capital?", "target_text": "Singapore City is the capital, regulated by MAS."},
    {"input_text": "What does SGD stand for?", "target_text": "SGD stands for Singapore Dollar, the official currency."},
    {"input_text": "Where is MAS located?", "target_text": "MAS is located in Singapore's financial district."},
    {"input_text": "What is Singapore known for?", "target_text": "Singapore is known as a financial hub with MAS oversight."},
    {"input_text": "How many banks are in Singapore?", "target_text": "Singapore has over 200 banks supervised by MAS."},
    {"input_text": "What does MAS regulate?", "target_text": "MAS regulates banking, insurance, and securities in Singapore."},
    {"input_text": "Why is Singapore important?", "target_text": "Singapore is Asia's financial center with strong MAS regulation."}
]

dataset = Dataset.from_list(simple_data)
print(f"\n✅ Dataset created: {len(dataset)} examples")


## 🔧 AGGRESSIVE LoRA Setup


In [ ]:
# Load model and save original for comparison
print("Loading Flan-T5-small...")
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# AGGRESSIVE LoRA config (KEY INSIGHT!)
print("\nSetting up AGGRESSIVE LoRA...")
lora_config = LoraConfig(
    r=32,  # MUCH higher rank (was 8)
    lora_alpha=64,  # MUCH higher alpha (was 16)
    target_modules=["q", "v", "k", "o"],  # More modules (was just q,v)
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✅ AGGRESSIVE LoRA applied!")
